## 1. Higher Order Functions

- Higher order functions are functions that accept another function as an argument, and return them as result
    - Kind of like a decorator in Python

- Like in Python, Scala treats function as **first-class values**
    - This means that functions can be passed as a value, and returned as a result

- This can give us some really flexible use cases, where we can compose higher order functions that take in functions as an input. 

- Imagine we have these functions below
    ```scala
        def sumDoubles(a: Int, b: Int): Int = 
          def double(a: Int): Int = {
            2*a
          }
          if a > b then 0 else double(a) + sumDoubles(a+1, b)
        
        def sumCubes(a: Int, b: Int): Int = 
          def cube(x: Int): Int = {
            x*x*x
          }
          if a > b then 0 else cube(a) + sumCubes(a+1, b)
        
        def sumFactorials(a: Int, b: Int): Int = 
          def factorial(x: Int): Int = {
            if x == 0 then 1 else x * factorial(x-1)
          }
          if a > b then 0 else factorial(a) + sumFactorials(a+1, b)
    ```

- Thinking through the above, these are all generalisations of a common pattern:
    $$\begin{aligned}
        \sum_{n=a}^{b} f(n)
    \end{aligned}$$

- So instead of implementing 3 different functions, why not just abstract the common pattern out?
    ```scala
        def double(a: Int): Int = {
          2*a
        }
        
        def cube(x: Int): Int = {
          x*x*x
        }

        def factorial(x: Int): Int = {
          if x == 0 then 1 else x * factorial(x-1)
        }

        def sumfunc(func: Int => Int, a: Int, b: Int): Int = {
            if a > b then 0
            else func(a) + sumfunc(func, a+1, b)
        }

        def sumfunc_With2Inputs(func: (Int, Int) => Int, a: Int, b: Int): Int = {
            if a > b then 0
            else func(a) + sumfunc(func, a+1, b)
        }

        def sumfunc_With2Inputs(func: (Int, Int) => Int, a: Int, b: Int): Int = {
            if a > b then 0
            else func(a) + sumfunc(func, a+1, b)
        }
    ```

- The new notation here is `func: Int => Int`, which indicates that `func` accepts a function that maps an Int to another Int. 
    - We have also shown how to implement this with a function that takes 2 inputs

### Anonymous Functions

- For cases when you are passing functions into functions, you will almost certainly end up with many small functions, which causes problems because
    - It pollutes your namespace
    - It is **tedious** to write and maintain

- Is there a way we can pass a function "on-demand" without naming it? Kind of like a lambda function in python?

- Scala lets you do this too!
    ```scala
        (x: Int) => x * x * x // (parameter) => body
        
        // same thing
        def cube(x: Int): Int = {
          x*x*x
        }
    ```

- So in fact, our earlier `sumfunc` can be used in a more concise way!!
    ```scala
        def sumCubes(a: Int, b: Int) = sumfunc(x => x*x*x, a, b)
    ```

### Exercise

- The sum function uses linear recursion. Write a tail-recursive version by
replacing the ???

    ```scala
        def sum(f: Int => Int, a: Int, b: Int): Int =
          def loop(a: Int, acc: Int): Int =
            if ??? then ???
            else loop(???, ???)
          loop(???, ???)
    ```

    ```scala
        def sum(f: Int => Int, a: Int, b: Int): Int =
          def loop(a: Int, acc: Int): Int =
            if a > b then acc
            else loop(a+1, acc+f(a))
          loop(a, 0)
    ```

## 2. Currying

- From the section above, we talked about using lambda functions to compose multiple functions together
    
    ```scala
        def sumfunc(func: Int => Int, a: Int, b: Int): Int = {
          if a > b then 0
          else func(a) + sumfunc(func, a+1, b)
        }

        def sumCubes(a: Int, b: Int) = sumfunc(x => x*x*x, a, b)
    ```

- This style presents some problems
    - `a` and `b` are unused by `sumfunc` (directly, anyway)
    - If we want to chain an arbtrary number of functions together, the syntax becomes unwieldy e.g. 
        ```scala 
            sumfunc(x => y => z => x*2*y*3*z, a, b)
        ```

- Let's do a little separation of concerns; we'll abstract `sumfunc` as a function that return a fuction
    ```scala
        def sumfunc(f: Int => Int): (Int, Int) => Int = {
          def composedFunc(a: Int, b: Int): Int = {
            if a > b then 0
            else f(a) + composedFunc(a+1, b)
          }
          composedFunc
        }
    ```

- By writing it this way, our `sumCubes` function can now be written as:
    ```scala
        def sumCubes = sumfunc(x => x * x * x)
    ```

- Yet, this approach still has problems, because we still need to define an intermediate function `sumCubes`!!
    - This causes the namespace pollution we previously discussed

- In actual fact, we can reduce the `sumfunc` definition even further by using a new syntax that Scala enables
    ```scala
        // takes in a function mapping Int => Int
        // you can use this to create a partial function, without supplying `a` and `b`!
        // i.e. val partialfunc = sum(func1); partialfunc(1,10)

        def sum(f: Int => Int)(a: Int, b: Int): Int = {
          if a > b then 0 else f(a) + sum(f)(a+1, b)
        }
    ```


- This can actually be avoided, if we apply this syntax!
    ```scala
        sumfunc(cube)(1, 10)
    ```

- The main value of this syntax is that it allows you to create the equivalent of Python's `partial` function natively! That is, I could define a partial function, and reuse it!
    ```scala
        def sumFunc(f: Int => Int)(a: Int, b: Int, baseCase: Int): Int = {
          def subFunc(a: Int, b: Int, baseCase: Int): Int = {
            if a > b then baseCase
            else f(a) + subFunc(a+1, b, baseCase)
          }
          subFunc
        }
    ```

- QUIZ: Given the function below, what is the type of sum?
    ```scala
        def sum(f: Int => Int)(a: Int, b: Int): Int = ...
    ```
    - `sum` returns an Int eventually

### Exercise

- Write a `product` function that calculates the product of the values of a function for the points on a given interval

- Write `factorial` in terms of `product`

- Can you write a more general function, which generalizes both `sum`
and `product`?

```scala
    def product(a: Int, b: Int): Int = {
      var res = 1
      for (i <- a to b) {
        res = res * i
      }
      res
    }

    def productRecursive(f: Int => Int, a: Int, b: Int): Int = {
      if a > b then 1
      else f(a) * productRecursive(f, a+1, b)
    }

    def productCurry(f: Int => Int)(a: Int, b: Int): Int = {
      if a > b then 1
      else f(a) * productCurry(f)(a+1, b)
    }

    def mapReduce(valueFunc: Int => Int, combineFunc: (Int, Int) => Int, baseCase: Int)(a: Int, b: Int): Int = {
      def recur(a: Int): Int = {
        if a > b then baseCase
        else combineFunc(valueFunc(a), recur(a+1))
      }
      recur(a)
    }

    def productMapReduce(f: Int => Int)(a: Int, b: Int) = {
      mapReduce(f, _ * _, 1)(a, b)
    }

    def sumMapReduce(f: Int => Int)(a: Int, b: Int) = {
      mapReduce(f, _ + _, 0)(a, b)
    }

    def factorial(x: Int): Int = {
      // product(1, x)
      // productRecursive(x => x, 1, x)
      // productCurry(x => x)(1, x)
      productMapReduce(x => x)(1, 10)
    }

```

## 3. Example of Higher Order Functions: Finding Fixed Point

- A fixed point of a function occurs if $f(x) = x$
    - For example, $\sin(0) = 0$
- For some functions, if we iteratively apply the function to an initial estimate, we will end up locating a fixed point! That is;
    - start with random value $a$
    - $f(a)$
    - $f(f(a))$
    - ...
    - this sequence approaches $x$

- Let's try first with this
  ```scala
    import scala.math.abs

    val tolerance = 0.0001

    def isCloseEnough(guess: Double, nextGuess: Double): Boolean = {
      abs((nextGuess - guess) / guess) < tolerance
    }

    def getNextGuess(candidate: Double)(guess: Double): Double = {
      candidate/guess
    }

    def getNextGuessDamping(candidate: Double)(guess: Double): Double = {
      (guess + candidate/guess) / 2
    }

    def fixedPoint(getNextGuessFunc: Double => Double, isCloseEnough: (Double, Double) => Boolean)(firstGuess: Double): Double = {
      def iterate(guess: Double): Double =
        val nextGuess = getNextGuessFunc(guess)
        println(guess + "||" + firstGuess + "||" + nextGuess)
        if isCloseEnough(guess, nextGuess) then nextGuess
        else iterate(nextGuess)
      iterate(firstGuess)
    }

    @main def sqrt(x: Double) = {
      // val res = fixedPoint(getNextGuess(x), isCloseEnough)(1.0)
      val res = fixedPoint(getNextGuessDamping(x), isCloseEnough)(1.0)
      println(res)
    }
  ```

- This is all sensible, but we have a problem. The `getNextGuess` function for `candidate=2` will cycle between returning 2 and 1 and will never terminate!

- To avoid this problem, we modify our candidate acquisition `getNextGuess` by taking an average of the current guess and the next guess! We'll call this `getNextGuessDamping`

- As you can see, the abiliy to pass functions into other functions can give us the ability to do some remarkable things!

## 5. Functions and Data

- Just as data objects are often expressed as `class` in Python, Scala offers the same notion of a `class`

- In this section, we'll design a package for doing arithmetic for rational numbers  

- Intuitively, the first thing we want to define is a class `Rational` that allows us to define a rational number. What happens when we don't? We end up with some really ugly function signatures
    ```scala
      def addRationalNumerator(n1: Int, d1: Int, n2: Int, d2: Int): Int = ...
      def addRationalDenominator(n1: Int, d1: Int, n2: Int, d2: Int): Int = ...
    ```

- Defining the `Rational` class. In Scala this does 2 things; creates a new **type** that you can use, and creates a new **constructor** for objects of this type
    ```scala
      class Rational(x: Int, y: Int):
        def numerator = x
        def denominator = y
    ```

- All elements that are defined by `class` are **objects**. 
  - An object can simply be created by calling the constructor of this class. 
  - We can access **members** (aka attributes) of the class by using the `.` operator
    ```scala
      val x = Rational(1,2)
      x.numerator // 1
      x.denominator // 2
    ```

- Having defined the `Rational` class, we can more easily implement functions that operate on objects of this class;
    ```scala
      def addRational(r1: Rational, r2: Rational): Rational = {
        Rational(
          (r1.numerator * r2.denominator) + (r2.numerator * r1.denominator),
          r1.denominator * r2.denominator
        )
      }

      def makeString(r: Rational): String = {
        s"${r.numerator}/${r.denominator}" //string interpolation, similar to f-string in Python
      }
    ```

- In fact, to better group functions together, we can actually add them to the class directly. When added this way, functions become class `method`s
    ```scala
      class Rational(x: Int, y: Int):
        def numerator = x
        def denominator = y

        def add(r: Rational) = {
          Rational(
            (numerator * r.denominator) + (r.numerator * denominator),
            denominator * r.denominator
          )
        }

        def multiply(r: Rational) = {
          Rational(
            numerator * r.numerator,
            denominator * r.denominator
          )
        }

        override def toString = s"${numerator}/${denominator}"

        val x = Rational(1,3)
        val y = Rational(5,7)
        val z = Rational(3,2)

        x.add(y).multiply(z)
    ```


## 6. Data Abstraction

- In this segment, let's try to extend the `Rational` class we wrote in the last section

### Adding a private method within a class

- You may have noticed that in the previous `Rational` implementation, we do not return the normalised rational number. That is, 10/6 is returned as is, not as 5/3

- Let's try adding this by dividing the numerator and denominator by their greatest common divisor `gcd`
  ```scala
    class Rational(x: Int, y: Int):
      private def gcd(x: Int, y: Int): Int = {
        if y == 0 then x
        else gcd(y, x % y)
      }

      private var gcdVal: Int = gcd(x, y)

      // def numerator: Int = x /gcdVal
      // def denominator: Int = y / gcdVal
      val numerator: Int = x /gcdVal
      val denominator: Int = y / gcdVal

      override def toString() = {
        s"${numerator}/${denominator}"
      }
  ```

- Notice a few things 
  - `gcd()` method is marked with a `private`, which blocks it from being used outside the class. Similar to Java!

  - `numerator` and `denominator` are changed from `def` to `val`. This is because `val` is evaluated only once, but `def` is evaluated each time it is called!

### Self Reference

- Similar to how a Python class allows you to reference a class method by using the keyword `self`, Scala lets you reference its own functions/attributes using the `this` keyword

  ```scala
    class Rational(x: Int, y: Int):
      private def gcd(x: Int, y: Int): Int = {
        if y == 0 then x
        else gcd(y, x % y)
      }

      private var gcdVal: Int = gcd(x, y)

      // def numerator: Int = x /gcdVal
      // def denominator: Int = y / gcdVal
      val numerator: Int = x /gcdVal
      val denominator: Int = y / gcdVal

      override def toString() = {
        s"${numerator}/${denominator}"
      }

      def less(that: Rational): Boolean = {
        this.numerator * that.denominator < that.numerator * this.denominator
      }     
      
      def max(that: Rational): Rational = {
        if this.less(that) then that else this
      }
  ```

### Preconditions vs Assertions

- Preconditions are enforced by the keyword `require`, while assertions are enforced by the keyword `assert`

- Both serve similar functions; that is to check that specific expectations are met

- However, `assert` can be switched off at run time, but `require` cannot

- Therefore, it is always recommended to use `require` whenever you want to ensure something MUST happen

  ```scala
      class Rational(x: Int, y: Int):
        ...
        
        require(y > 0, "cannot have 0 denominator")
        val numerator: Int = x /gcdVal
        val denominator: Int = y / gcdVal

    ```

### Constructors

- We said previously that defining a class adds both a type and a constructor to the namespace by default

- HOWEVER, it is possible to add a custom constructor, known in Scala as an **auxiliary constructor**
  - This is done by adding a method called `this(...)` with the relevant signature

  ```scala
    class Rational(x: Int, y: Int):
      def this(x: Int) = this(x, 1)
  ```

### End Markers

- While this is not strictly needed, it is sometimes useful to add the `end` marker to your classes or methods

- This enhances readability

### Exercise

- Modify the Rational class so that rational numbers are kept unsimplified internally, but the simplification is applied when numbers are converted to strings.

- Do clients observe the same behavior when interacting with the rational class?
  - Yes assuming no overflow


- See `6-slides.scala`

## 7. Evaluation and Operators

- Writing a class also entails its own order of operations, and evaluation logic. We previously saw something similar in Module 1 7-slides.pdf

- I don't think learning the notation of the substitution process is super important, so will just cover the 2 things I found important this week;
  - `extension` methods
  - `infix` methods

### Using Extensions

- If you keep adding methods to the class, you end up with a massive ugly class. Which, from a readability POV, is not ideal

- Scala lets you create an `extension` to a class that can be kept logically separated from the main class
  - This lets you define non-core methods for the class away from the class
  - BUT it assumes they do not need to operate on stuff that is internal to the class
  - You also cannot use an `extension` to overwrite any existing methods

  ```scala
    extension (r: Rational)
      def min(s: Rational): Rational = if s.less(r) then s else r
      def abs: Rational = Rational(r.numer.abs, r.denom)
  ```


### Infix Operators

- In our `Rational` class, we define a method `add`

- This has the same behaviour as `+`, but it is not recognised as a `+` operation

- This poorly designed interface can lead to confusion 
  - for `Int`, we can do `x + y`
  - for `Rational`, we have to do `x.add(y)`

- In Scala, we can eliminate this difference in interface by **infix**-ing a new operator

- To do this, we can define custom behaviour for operators
  ```scala
    extension (x: Rational)
      def + (y: Rational): Rational = x.add(y)
      def * (y: Rational): Rational = x.mul(y)
      ...
  ```

  - Why is this allowed? Because `+` / `*` etc operators are valid identifiers! 

- Next, we use the `infix` keyword to declare the behaviour tied to this operator
  ```scala
    extension (x: Rational)
      infix def min(that: Rational): Rational = ...
  ```

- This allows us to do
  ```scala
    r + s
    r min s
  ```


- When making composite operators, its precedence is determined by its first character 
  - That is `*=` will always be resolved before `+=`

- The order of precedence is (from lowest to highest)
  - (all letters)
  - `|`
  - `^`
  - `&`
  - `< >`
  - `= !`
  - `:`
  - `+ -`
  - `* / %`
  - (any other special chars)

### Exercise

- Provide a fully parenthesized version of: 
  - `a + b ^? c ?^ d less a ==> b | c`

- Every binary operation needs to be put into parentheses, but the structure of the expression should not change.

- Referencing order of precedence above:

    - `(((a + b) ^? (c ?^ d)) less ((a ==> b) | c))`